# Pydantic for Data Modeling and Validation

This notebook demonstrates Pydantic's powerful features for data modeling, validation, and serialization - essential skills for building robust data applications in Databricks and Streamlit.

## Learning Objectives
- Understand Pydantic models and their benefits
- Learn data validation and type safety
- Master serialization and deserialization
- Explore advanced Pydantic features
- Build models for real-world scenarios

In [ ]:
# Install required packages (run this if Pydantic is not installed)
# !pip install pydantic[email] python-dateutil

## 1. Introduction to Pydantic

Pydantic provides data validation and parsing using Python type annotations.

In [ ]:
from pydantic import BaseModel, Field, validator
from typing import Optional, List, Dict, Union
from datetime import datetime, date
import json

# Simple Pydantic model
class Employee(BaseModel):
    id: int
    name: str
    email: str
    salary: float
    is_active: bool = True
    hire_date: Optional[date] = None

# Create an employee instance
employee_data = {
    "id": 1,
    "name": "Alice Johnson",
    "email": "alice@company.com",
    "salary": 75000.50,
    "hire_date": "2023-01-15"
}

employee = Employee(**employee_data)
print(f"Employee: {employee}")
print(f"Employee name: {employee.name}")
print(f"Employee type: {type(employee)}")

### Benefits of Pydantic Models

1. **Type Safety**: Automatic type conversion and validation
2. **Data Validation**: Built-in and custom validators
3. **IDE Support**: Better autocomplete and error detection
4. **Serialization**: Easy JSON export/import
5. **Documentation**: Self-documenting code

In [ ]:
# Automatic type conversion
employee2 = Employee(
    id="2",  # String gets converted to int
    name="Bob Smith",
    email="bob@company.com",
    salary="68000",  # String gets converted to float
    is_active="true"  # String gets converted to bool
)

print(f"ID type: {type(employee2.id)} = {employee2.id}")
print(f"Salary type: {type(employee2.salary)} = {employee2.salary}")
print(f"Active type: {type(employee2.is_active)} = {employee2.is_active}")

## 2. Data Validation

Pydantic automatically validates data and provides clear error messages:

In [ ]:
from pydantic import ValidationError

# Test validation errors
invalid_data_cases = [
    {"id": "not_a_number", "name": "Test", "email": "test@email.com", "salary": 50000},
    {"id": 1, "name": "", "email": "invalid-email", "salary": -1000},
    {"id": 1, "name": "Test", "email": "test@email.com"} # Missing required salary
]

for i, invalid_data in enumerate(invalid_data_cases, 1):
    try:
        employee = Employee(**invalid_data)
        print(f"Case {i}: Valid - {employee}")
    except ValidationError as e:
        print(f"Case {i}: Validation Error:")
        for error in e.errors():
            print(f"  - {error['loc'][0]}: {error['msg']}")
        print()

## 3. Field Validation and Constraints

Use the `Field` function to add constraints and metadata:

In [ ]:
from pydantic import BaseModel, Field, EmailStr, validator
from typing import Optional
from datetime import date

class EmployeeV2(BaseModel):
    id: int = Field(..., gt=0, description="Employee ID must be positive")
    name: str = Field(..., min_length=2, max_length=100, description="Employee full name")
    email: EmailStr = Field(..., description="Valid email address")
    salary: float = Field(..., ge=0, le=1000000, description="Annual salary in USD")
    department: str = Field(..., regex=r'^(Engineering|Marketing|Sales|HR|Finance)$')
    is_active: bool = True
    hire_date: Optional[date] = None
    
    class Config:
        # Configuration for the model
        json_encoders = {
            date: lambda dt: dt.isoformat()
        }
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Alice Johnson",
                "email": "alice@company.com",
                "salary": 75000,
                "department": "Engineering",
                "hire_date": "2023-01-15"
            }
        }

# Test the enhanced model
valid_employee = EmployeeV2(
    id=1,
    name="Alice Johnson",
    email="alice@company.com",
    salary=75000,
    department="Engineering",
    hire_date="2023-01-15"
)

print(f"Valid employee: {valid_employee}")
print(f"JSON representation: {valid_employee.json(indent=2)}")

## 4. Custom Validators

Create custom validation logic for complex business rules:

In [ ]:
from datetime import date, datetime
from typing import Optional

class EmployeeV3(BaseModel):
    id: int = Field(gt=0)
    name: str = Field(min_length=2, max_length=100)
    email: str
    salary: float = Field(ge=0)
    department: str
    hire_date: Optional[date] = None
    performance_rating: Optional[float] = Field(None, ge=1.0, le=5.0)
    
    @validator('email')
    def validate_email(cls, v):
        """Custom email validation"""
        if '@' not in v or '.' not in v.split('@')[1]:
            raise ValueError('Invalid email format')
        return v.lower()  # Normalize to lowercase
    
    @validator('name')
    def validate_name(cls, v):
        """Normalize name format"""
        return ' '.join(word.capitalize() for word in v.split())
    
    @validator('hire_date')
    def validate_hire_date(cls, v):
        """Ensure hire date is not in the future"""
        if v and v > date.today():
            raise ValueError('Hire date cannot be in the future')
        return v
    
    @validator('salary')
    def validate_salary_range(cls, v, values):
        """Validate salary based on department"""
        department = values.get('department', '')
        min_salaries = {
            'Engineering': 60000,
            'Marketing': 45000,
            'Sales': 40000,
            'HR': 50000,
            'Finance': 55000
        }
        
        min_salary = min_salaries.get(department, 30000)
        if v < min_salary:
            raise ValueError(f'Salary for {department} must be at least ${min_salary:,}')
        return v

# Test custom validators
test_employee = EmployeeV3(
    id=1,
    name="alice JOHNSON",  # Will be normalized
    email="Alice@Company.COM",  # Will be lowercased
    salary=75000,
    department="Engineering",
    hire_date="2023-01-15",
    performance_rating=4.2
)

print(f"Normalized employee: {test_employee}")
print(f"Name: {test_employee.name}")
print(f"Email: {test_employee.email}")

## 5. Nested Models and Complex Structures

Build complex data structures with nested Pydantic models:

In [ ]:
from typing import List, Optional, Dict, Any
from enum import Enum

class DepartmentType(str, Enum):
    ENGINEERING = "Engineering"
    MARKETING = "Marketing"
    SALES = "Sales"
    HR = "HR"
    FINANCE = "Finance"

class Address(BaseModel):
    street: str
    city: str
    state: str = Field(max_length=2)  # US state code
    zip_code: str = Field(regex=r'^\d{5}(-\d{4})?$')  # US zip code format

class Skill(BaseModel):
    name: str
    level: str = Field(regex=r'^(Beginner|Intermediate|Advanced|Expert)$')
    years_experience: float = Field(ge=0, le=50)

class Employee(BaseModel):
    id: int = Field(gt=0)
    name: str = Field(min_length=2, max_length=100)
    email: str
    salary: float = Field(ge=0)
    department: DepartmentType
    is_active: bool = True
    hire_date: Optional[date] = None
    address: Optional[Address] = None
    skills: List[Skill] = []
    manager_id: Optional[int] = None
    metadata: Dict[str, Any] = {}
    
    @validator('email')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('Invalid email')
        return v.lower()

# Create a complex employee record
employee_data = {
    "id": 1,
    "name": "Alice Johnson",
    "email": "alice@company.com",
    "salary": 95000,
    "department": "Engineering",
    "hire_date": "2022-03-15",
    "address": {
        "street": "123 Tech Street",
        "city": "San Francisco",
        "state": "CA",
        "zip_code": "94105"
    },
    "skills": [
        {"name": "Python", "level": "Expert", "years_experience": 5.5},
        {"name": "Machine Learning", "level": "Advanced", "years_experience": 3.0},
        {"name": "SQL", "level": "Advanced", "years_experience": 4.0}
    ],
    "manager_id": 5,
    "metadata": {
        "security_clearance": "Level 2",
        "remote_work_eligible": True,
        "last_review_date": "2024-01-15"
    }
}

employee = Employee(**employee_data)
print(f"Employee created successfully: {employee.name}")
print(f"Department: {employee.department}")
print(f"Skills: {[f'{s.name} ({s.level})' for s in employee.skills]}")
print(f"Address: {employee.address.city}, {employee.address.state}")

## 6. Serialization and Deserialization

Convert between Pydantic models and various data formats:

In [ ]:
# JSON serialization
json_str = employee.json(indent=2)
print("JSON representation:")
print(json_str)

# Dictionary representation
dict_repr = employee.dict()
print(f"\nDictionary keys: {list(dict_repr.keys())}")

# Exclude certain fields
public_data = employee.dict(exclude={'salary', 'metadata'})
print(f"\nPublic data (no salary/metadata): {list(public_data.keys())}")

# Include only specific fields
summary = employee.dict(include={'name', 'department', 'skills'})
print(f"\nSummary: {summary['name']} - {summary['department']}")

In [ ]:
# Parse from JSON
json_data = '''
{
  "id": 2,
  "name": "Bob Smith",
  "email": "bob@company.com",
  "salary": 80000,
  "department": "Marketing",
  "skills": [
    {"name": "Digital Marketing", "level": "Expert", "years_experience": 4.0},
    {"name": "Analytics", "level": "Advanced", "years_experience": 2.5}
  ]
}
'''

bob = Employee.parse_raw(json_data)
print(f"Parsed from JSON: {bob.name} in {bob.department}")
print(f"Skills: {[s.name for s in bob.skills]}")

# Parse from dictionary
dict_data = {
    "id": 3,
    "name": "Charlie Brown",
    "email": "charlie@company.com",
    "salary": 70000,
    "department": "Sales"
}

charlie = Employee(**dict_data)
print(f"\nParsed from dict: {charlie.name}")

## 7. Model Configuration and Advanced Features

Customize model behavior with configuration options:

In [ ]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class AuditableModel(BaseModel):
    created_at: datetime = Field(default_factory=datetime.now)
    updated_at: Optional[datetime] = None
    created_by: Optional[str] = None
    
    class Config:
        # Allow extra fields
        extra = "forbid"  # or "allow" or "ignore"
        # Use enum values instead of enum objects
        use_enum_values = True
        # Validate assignments after object creation
        validate_assignment = True
        # Custom JSON encoders
        json_encoders = {
            datetime: lambda dt: dt.isoformat()
        }

class Product(AuditableModel):
    id: int = Field(gt=0)
    name: str = Field(min_length=1, max_length=200)
    price: float = Field(gt=0)
    category: str
    in_stock: bool = True
    description: Optional[str] = None
    
    @validator('price')
    def round_price(cls, v):
        return round(v, 2)

# Create a product with audit fields
product = Product(
    id=1,
    name="Laptop Pro",
    price=1299.999,  # Will be rounded
    category="Electronics",
    description="High-performance laptop for professionals",
    created_by="system"
)

print(f"Product: {product.name}")
print(f"Price (rounded): ${product.price}")
print(f"Created at: {product.created_at}")

# Test validate_assignment
try:
    product.price = -100  # This should raise an error
except ValidationError as e:
    print(f"\nValidation error on assignment: {e}")

# JSON representation with custom encoder
print(f"\nJSON: {product.json()}")

## 8. Working with Lists and Bulk Operations

Handle collections of models efficiently:

In [ ]:
from typing import List
from pydantic import parse_obj_as

# Sample employee data
employees_data = [
    {
        "id": 1,
        "name": "Alice Johnson",
        "email": "alice@company.com",
        "salary": 95000,
        "department": "Engineering",
        "skills": [{"name": "Python", "level": "Expert", "years_experience": 5}]
    },
    {
        "id": 2,
        "name": "Bob Smith",
        "email": "bob@company.com",
        "salary": 80000,
        "department": "Marketing",
        "skills": [{"name": "Analytics", "level": "Advanced", "years_experience": 3}]
    },
    {
        "id": 3,
        "name": "Charlie Brown",
        "email": "charlie@company.com",
        "salary": 70000,
        "department": "Sales",
        "skills": []
    }
]

# Parse list of employees
employees = parse_obj_as(List[Employee], employees_data)
print(f"Parsed {len(employees)} employees")

# Process the list
for emp in employees:
    skill_count = len(emp.skills)
    print(f"- {emp.name} ({emp.department}): {skill_count} skills")

# Calculate statistics
total_salary = sum(emp.salary for emp in employees)
avg_salary = total_salary / len(employees)
print(f"\nTotal payroll: ${total_salary:,.2f}")
print(f"Average salary: ${avg_salary:,.2f}")

# Export to JSON
employees_json = [emp.dict() for emp in employees]
print(f"\nExported {len(employees_json)} employee records to JSON format")

## 9. Integration with Databricks and Data Processing

Use Pydantic models for data processing pipelines:

In [ ]:
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field, validator
from datetime import date
import pandas as pd

class DataRow(BaseModel):
    """Base class for data processing rows"""
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary for DataFrame creation"""
        return self.dict()
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]):
        """Create instance from dictionary"""
        return cls(**data)

class SalesRecord(DataRow):
    transaction_id: str
    customer_id: int = Field(gt=0)
    product_id: int = Field(gt=0)
    quantity: int = Field(gt=0)
    unit_price: float = Field(gt=0)
    discount: float = Field(ge=0, le=1.0)  # Percentage as decimal
    sale_date: date
    sales_rep_id: Optional[int] = None
    region: str
    
    @property
    def total_amount(self) -> float:
        """Calculate total amount after discount"""
        gross = self.quantity * self.unit_price
        return gross * (1 - self.discount)
    
    @validator('transaction_id')
    def validate_transaction_id(cls, v):
        if not v.startswith('TXN'):
            raise ValueError('Transaction ID must start with TXN')
        return v.upper()

# Sample sales data
sales_data = [
    {
        "transaction_id": "txn001",
        "customer_id": 1001,
        "product_id": 2001,
        "quantity": 2,
        "unit_price": 29.99,
        "discount": 0.1,
        "sale_date": "2024-01-15",
        "sales_rep_id": 501,
        "region": "North"
    },
    {
        "transaction_id": "txn002",
        "customer_id": 1002,
        "product_id": 2002,
        "quantity": 1,
        "unit_price": 49.99,
        "discount": 0.0,
        "sale_date": "2024-01-16",
        "region": "South"
    }
]

# Process sales records
sales_records = [SalesRecord(**record) for record in sales_data]

# Calculate metrics
total_revenue = sum(record.total_amount for record in sales_records)
print(f"Total Revenue: ${total_revenue:.2f}")

for record in sales_records:
    print(f"- {record.transaction_id}: ${record.total_amount:.2f} ({record.region})")

# Convert to DataFrame (useful for Databricks)
df_data = [record.to_dict() for record in sales_records]
# Add calculated field
for i, record in enumerate(sales_records):
    df_data[i]['total_amount'] = record.total_amount

print(f"\nDataFrame-ready data: {len(df_data)} records")
print(f"Columns: {list(df_data[0].keys())}")

## 10. Error Handling and Debugging

Best practices for handling validation errors:

In [ ]:
from pydantic import ValidationError
from typing import List, Dict, Any, Tuple

def safe_parse_employee(data: Dict[str, Any]) -> Tuple[Optional[Employee], Optional[str]]:
    """Safely parse employee data with error handling"""
    try:
        employee = Employee(**data)
        return employee, None
    except ValidationError as e:
        # Create user-friendly error message
        errors = []
        for error in e.errors():
            field = '.'.join(str(x) for x in error['loc'])
            message = error['msg']
            errors.append(f"{field}: {message}")
        return None, "; ".join(errors)

def batch_parse_employees(data_list: List[Dict[str, Any]]) -> Tuple[List[Employee], List[Dict[str, Any]]]:
    """Parse a batch of employees, separating valid and invalid records"""
    valid_employees = []
    invalid_records = []
    
    for i, data in enumerate(data_list):
        employee, error = safe_parse_employee(data)
        if employee:
            valid_employees.append(employee)
        else:
            invalid_records.append({
                'index': i,
                'data': data,
                'errors': error
            })
    
    return valid_employees, invalid_records

# Test with mixed valid/invalid data
mixed_data = [
    {  # Valid
        "id": 1,
        "name": "Alice Johnson",
        "email": "alice@company.com",
        "salary": 75000,
        "department": "Engineering"
    },
    {  # Invalid - missing required fields
        "id": 2,
        "name": "Bob Smith"
    },
    {  # Invalid - bad data types
        "id": "not_a_number",
        "name": "",
        "email": "invalid-email",
        "salary": -1000,
        "department": "NonExistentDept"
    },
    {  # Valid
        "id": 3,
        "name": "Charlie Brown",
        "email": "charlie@company.com",
        "salary": 68000,
        "department": "Sales"
    }
]

valid_employees, invalid_records = batch_parse_employees(mixed_data)

print(f"Successfully parsed: {len(valid_employees)} employees")
print(f"Failed to parse: {len(invalid_records)} records")

print("\nValid employees:")
for emp in valid_employees:
    print(f"  - {emp.name} ({emp.department})")

print("\nInvalid records:")
for record in invalid_records:
    print(f"  - Record {record['index']}: {record['errors']}")

## 11. Model Inheritance and Composition

Build complex models using inheritance and composition:

In [ ]:
from abc import ABC
from typing import Optional, Union

# Base models
class Person(BaseModel):
    """Base person model"""
    name: str = Field(min_length=1)
    email: str
    phone: Optional[str] = None
    
    @validator('email')
    def validate_email(cls, v):
        if '@' not in v:
            raise ValueError('Invalid email')
        return v.lower()

# Specialized models
class Employee(Person):
    """Employee model extending Person"""
    employee_id: int = Field(gt=0)
    salary: float = Field(gt=0)
    department: DepartmentType
    hire_date: date
    is_active: bool = True

class Customer(Person):
    """Customer model extending Person"""
    customer_id: int = Field(gt=0)
    registration_date: date
    loyalty_points: int = Field(ge=0, default=0)
    preferred_contact: str = Field(regex=r'^(email|phone|mail)$', default='email')

class Contractor(Person):
    """Contractor model extending Person"""
    contractor_id: int = Field(gt=0)
    hourly_rate: float = Field(gt=0)
    contract_start: date
    contract_end: Optional[date] = None
    skills: List[str] = []

# Create instances
employee = Employee(
    name="Alice Johnson",
    email="alice@company.com",
    employee_id=1001,
    salary=85000,
    department=DepartmentType.ENGINEERING,
    hire_date="2023-01-15"
)

customer = Customer(
    name="Bob Smith",
    email="bob@customer.com",
    customer_id=2001,
    registration_date="2024-01-01",
    loyalty_points=150
)

contractor = Contractor(
    name="Charlie Brown",
    email="charlie@freelancer.com",
    contractor_id=3001,
    hourly_rate=75.0,
    contract_start="2024-02-01",
    skills=["Python", "Data Analysis", "Machine Learning"]
)

print(f"Employee: {employee.name} - {employee.department}")
print(f"Customer: {customer.name} - {customer.loyalty_points} points")
print(f"Contractor: {contractor.name} - ${contractor.hourly_rate}/hr")

# Polymorphic usage
people = [employee, customer, contractor]
print("\nAll people:")
for person in people:
    print(f"  - {person.name} ({type(person).__name__})")

## 12. Integration Patterns for Streamlit

Use Pydantic models to validate user input in Streamlit apps:

In [ ]:
from typing import Dict, Any, List
from pydantic import ValidationError

class FormValidator:
    """Helper class for validating form data in Streamlit"""
    
    @staticmethod
    def validate_and_create(model_class, form_data: Dict[str, Any]) -> Tuple[Any, List[str]]:
        """Validate form data and create model instance"""
        try:
            instance = model_class(**form_data)
            return instance, []
        except ValidationError as e:
            errors = []
            for error in e.errors():
                field = '.'.join(str(x) for x in error['loc'])
                message = error['msg']
                errors.append(f"{field.title()}: {message}")
            return None, errors
    
    @staticmethod
    def get_model_schema(model_class) -> Dict[str, Any]:
        """Get model schema for building forms"""
        return model_class.schema()

# Example form data validation
class UserRegistration(BaseModel):
    username: str = Field(min_length=3, max_length=20, regex=r'^[a-zA-Z0-9_]+$')
    email: str
    age: int = Field(ge=18, le=120)
    terms_accepted: bool
    newsletter_signup: bool = False
    
    @validator('email')
    def validate_email(cls, v):
        if '@' not in v or '.' not in v:
            raise ValueError('Please enter a valid email address')
        return v.lower()
    
    @validator('terms_accepted')
    def validate_terms(cls, v):
        if not v:
            raise ValueError('You must accept the terms and conditions')
        return v

# Simulate form submission
form_submissions = [
    {  # Valid
        "username": "alice123",
        "email": "alice@example.com",
        "age": 28,
        "terms_accepted": True,
        "newsletter_signup": True
    },
    {  # Invalid
        "username": "ab",  # Too short
        "email": "invalid-email",  # Invalid format
        "age": 16,  # Too young
        "terms_accepted": False,  # Must be true
        "newsletter_signup": False
    }
]

print("Form Validation Results:")
for i, form_data in enumerate(form_submissions, 1):
    user, errors = FormValidator.validate_and_create(UserRegistration, form_data)
    
    if user:
        print(f"\nSubmission {i}: SUCCESS")
        print(f"  User: {user.username} ({user.email})")
        print(f"  Newsletter: {'Yes' if user.newsletter_signup else 'No'}")
    else:
        print(f"\nSubmission {i}: FAILED")
        for error in errors:
            print(f"  - {error}")

# Get schema for building forms
schema = FormValidator.get_model_schema(UserRegistration)
print(f"\nModel has {len(schema['properties'])} fields:")
for field_name, field_info in schema['properties'].items():
    field_type = field_info.get('type', 'unknown')
    print(f"  - {field_name}: {field_type}")

## 13. Performance and Best Practices

Tips for using Pydantic efficiently:

In [ ]:
import time
from typing import List

# Performance comparison: dict vs Pydantic model
def create_dict_records(n: int) -> List[Dict[str, Any]]:
    """Create records using plain dictionaries"""
    records = []
    for i in range(n):
        record = {
            "id": i + 1,
            "name": f"Person {i + 1}",
            "email": f"person{i + 1}@example.com",
            "salary": 50000 + (i * 1000)
        }
        records.append(record)
    return records

def create_pydantic_records(n: int) -> List[Employee]:
    """Create records using Pydantic models"""
    records = []
    for i in range(n):
        record = Employee(
            id=i + 1,
            name=f"Person {i + 1}",
            email=f"person{i + 1}@example.com",
            salary=50000 + (i * 1000),
            department=DepartmentType.ENGINEERING
        )
        records.append(record)
    return records

# Benchmark creation time
n = 1000

# Dict creation
start_time = time.time()
dict_records = create_dict_records(n)
dict_time = time.time() - start_time

# Pydantic creation
start_time = time.time()
pydantic_records = create_pydantic_records(n)
pydantic_time = time.time() - start_time

print(f"Performance Comparison (n={n}):")
print(f"Dict creation: {dict_time:.4f} seconds")
print(f"Pydantic creation: {pydantic_time:.4f} seconds")
print(f"Pydantic overhead: {pydantic_time / dict_time:.2f}x")

# Best practices
print("\n" + "="*50)
print("PYDANTIC BEST PRACTICES")
print("="*50)

practices = [
    "1. Use type hints consistently",
    "2. Implement custom validators for business rules",
    "3. Use Field() for constraints and documentation",
    "4. Handle ValidationError gracefully",
    "5. Use Config class for model customization",
    "6. Consider performance for high-volume operations",
    "7. Use parse_obj_as() for list parsing",
    "8. Implement proper error messages for users",
    "9. Use model inheritance for related models",
    "10. Document models with docstrings and examples"
]

for practice in practices:
    print(practice)

## Summary and Next Steps

This notebook covered comprehensive Pydantic usage for data modeling:

### Key Concepts Learned:
1. **Basic Models**: Creating and using Pydantic models
2. **Validation**: Built-in and custom validators
3. **Serialization**: JSON and dict conversion
4. **Advanced Features**: Inheritance, composition, and configuration
5. **Error Handling**: Graceful validation error management
6. **Integration**: Use with Streamlit and data processing
7. **Performance**: Best practices for efficiency

### Benefits for Your Projects:
- **Databricks**: Validate data before processing
- **Streamlit**: Ensure user input integrity
- **APIs**: Consistent data models
- **Data Pipelines**: Type-safe data transformations

### Next Steps:
1. Practice with your own data models
2. Integrate with Databricks workflows
3. Build Streamlit forms with validation
4. Explore Pydantic V2 features
5. Learn about FastAPI integration